# edge-connect with differentiable augmentation
edge-connect: [knazeri/edge-connect](https://github.com/knazeri/edge-connect)

Yukariins fork: [Yukariin/edge-connect](https://github.com/Yukariin/edge-connect)

Differentiable Augmentation: [mit-han-lab/data-efficient-gans](https://github.com/mit-han-lab/data-efficient-gans)

My fork: [styler00dollar/Colab-edge-connect](https://github.com/styler00dollar/Colab-edge-connect)

In [ ]:
# check gpu
!nvidia-smi

In [ ]:
#@title Install
!git clone https://github.com/styler00dollar/Colab-edge-connect
%cd Colab-edge-connect
!pip install -r requirements.txt
!pip install tensorboardX

# Test with pre-trained models

In [ ]:
#@title Download models
%cd /content/Colab-edge-connect
!pip install gdown
!mkdir checkpoint_places
%cd checkpoint_places
# /checkpoints broken
# places
!gdown --id 1gesVuuYMtlWSQRR2JE5eO0QZHskYRfqv
!gdown --id 1_oYnmK7kppXqka9UUsHrZB4gWE4ouSgT
!gdown --id 1M-r_ds4VZJnUqViDMofd4-Fy8-q2aeKJ
!gdown --id 1G8lXquU3eREfs8KorFpFC8N4YmTQRksF
%cd ..
!mkdir checkpoint_celeba
%cd checkpoint_celeba
!gdown --id 1wy0pEaXTqmya2yeLwWFmTBf4ICexCdce
!gdown --id 1hqZRjnqZBGnSTtGJRHXEvvdGVICUGa7u
!gdown --id 17FemN4FAKpS5-8Dos582IrOiSCZNDOAO
!gdown --id 15mH1ZHMf83q3woBHFELr_TptSRGc5g5j
%cd ..
!mkdir checkpoint_street
%cd checkpoint_street
!gdown --id 1ORF2uN4lB3F6YndPm1ny8VIDrsWQBwUS
!gdown --id 1EwHK8YjcpO-X3xhmeo2dtqGvtY5vOMMj
!gdown --id 1AWxB8AwTOrlOmAUho3IQQlmawtp3y8gZ
!gdown --id 12Ua8oQwk0iLdYgrb08bqBhfyiBIumQEK

In [ ]:
#@title dummy config
%%writefile /content/Colab-edge-connect/config.yml
MODE: 1             # 1: train, 2: test, 3: eval
MODEL: 1            # 1: edge model, 2: inpaint model, 3: edge-inpaint model, 4: joint model
MASK: 3             # 1: random block, 2: half, 3: external, 4: (external, random block), 5: (external, random block, half)
EDGE: 1             # 1: canny, 2: external
NMS: 1              # 0: no non-max-suppression, 1: applies non-max-suppression on the external edges by multiplying by Canny
SEED: 10            # random seed
GPU: [0]            # list of gpu ids
DEBUG: 0            # turns on debugging mode
VERBOSE: 0          # turns on verbose mode in the output console

TRAIN_FLIST: ./datasets/places2_train.flist
VAL_FLIST: ./datasets/places2_val.flist
TEST_FLIST: ./datasets/places2_test.flist

TRAIN_EDGE_FLIST: ./datasets/places2_edges_train.flist
VAL_EDGE_FLIST: ./datasets/places2_edges_val.flist
TEST_EDGE_FLIST: ./datasets/places2_edges_test.flist

TRAIN_MASK_FLIST: ./datasets/masks_train.flist
VAL_MASK_FLIST: ./datasets/masks_val.flist
TEST_MASK_FLIST: ./datasets/masks_test.flist

LR: 0.0001                    # learning rate
D2G_LR: 0.1                   # discriminator/generator learning rate ratio
BETA1: 0.0                    # adam optimizer beta1
BETA2: 0.9                    # adam optimizer beta2
BATCH_SIZE: 8                 # input batch size for training
INPUT_SIZE: 256               # input image size for training 0 for original size
SIGMA: 2                      # standard deviation of the Gaussian filter used in Canny edge detector (0: random, -1: no edge)
MAX_ITERS: 2e6                # maximum number of iterations to train the model

EDGE_THRESHOLD: 0.5           # edge detection threshold
L1_LOSS_WEIGHT: 1             # l1 loss weight
FM_LOSS_WEIGHT: 10            # feature-matching loss weight
STYLE_LOSS_WEIGHT: 250        # style loss weight
CONTENT_LOSS_WEIGHT: 0.1      # perceptual loss weight
INPAINT_ADV_LOSS_WEIGHT: 0.1  # adversarial loss weight

GAN_LOSS: nsgan               # nsgan | lsgan | hinge
GAN_POOL_SIZE: 0              # fake images pool size

SAVE_INTERVAL: 1000           # how many iterations to wait before saving model (0: never)
SAMPLE_INTERVAL: 1000         # how many iterations to wait before sampling (0: never)
SAMPLE_SIZE: 12               # number of images to sample
EVAL_INTERVAL: 0              # how many iterations to wait before model evaluation (0: never)
LOG_INTERVAL: 10              # how many iterations to wait before logging training status (0: never)

Currently default paths are ```/content/image.png``` and ```/content/mask.png```. Currently it's not supported that you change paths.

In [ ]:
#@title Image and mask need to be dividable by 4, this code does fix wrong images 
import cv2
import numpy
path_inpainting = '/content/image.png' #@param {type:"string"}
path_mask = '/content/mask.png' #@param {type:"string"}
image=cv2.imread(path_mask)
image_size0 = numpy.floor(image.shape[0]/4)
image_size1 = numpy.floor(image.shape[1]/4)
image=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
ret,image=cv2.threshold(image,254,255,cv2.THRESH_BINARY)
image = cv2.resize(image, (int(image_size1*4), int(image_size0*4)), cv2.INTER_NEAREST)
cv2.imwrite(path_mask, image)

image=cv2.imread(path_inpainting)
image = cv2.resize(image, (int(image_size1*4), int(image_size0*4)), cv2.INTER_NEAREST)
cv2.imwrite(path_inpainting, image)

In [ ]:
#@title print shape
import cv2
image = cv2.imread(path_inpainting)
print(image.shape)
image = cv2.imread(path_mask)
print(image.shape)

In [ ]:
#@title Test Inpainting (result will be ```image.png```, the same filename you used as input)
%cd /content/Colab-edge-connect
!python test.py \
  --model 3 \
  --checkpoints /content/Colab-edge-connect/checkpoint_places \
  --input /content/image.png \
  --mask /content/mask.png \
  --output /content/

# Training

Interesting stuff:
- New pytorch versions won't work. Pytorch 1.0 makes problems. Pytorch 1.1 seems to work fine.
- The ```.tflist``` simply lists filepaths for images.
- It supports blocks as inpainting method, but random/custom masks need to be manually downloaded and input with a ```.tflist``` as well. Two example datasets are linked in the original github.
- [Model 4 is not recommended](https://github.com/knazeri/edge-connect/issues/144). You should probably use model 3.
- [Resuming and using a model as pretrained is being done by simply starting training while the models are in the specified checkpoint path.](https://github.com/knazeri/edge-connect/issues/54)

In [ ]:
#@title Create empty folders
!mkdir /content/training-checkpoints/
!mkdir /content/train/
!mkdir /content/val/
!mkdir /content/mask_train/
!mkdir /content/mask_val/

Input all your data..

In [ ]:
#@title modify paths inside prepare.py to create file list
%%writefile /content/Colab-edge-connect/prepare.py
#!/usr/bin/python

import argparse
import os
from random import shuffle

parser = argparse.ArgumentParser()
parser.add_argument('--is_shuffled', default='1', type=int,
                    help='Needed to shuffle')

if __name__ == "__main__":

    args = parser.parse_args()

    train_filename = 'train.flist'
    validation_filename = 'val.flist'

    train_path = '/home/path/'
    val_path = '/home/path/'

    training_file_names = []
    validation_file_names = []

    training_folder = os.listdir(train_path)

    for training_item in training_folder:
        training_item = train_path + "/" + training_item
        training_file_names.append(training_item)

    validation_folder = os.listdir(val_path)

    for validation_item in validation_folder:
        validation_item = val_path + "/" + validation_item
        validation_file_names.append(validation_item)

    # print all file paths
    for i in training_file_names:
        print(i)
    for i in validation_file_names:
        print(i)

    # This would print all the files and directories

    # shuffle file names if set
    if args.is_shuffled == 1:
        shuffle(training_file_names)
        shuffle(validation_file_names)

    # make output file if not existed
    if not os.path.exists(train_filename):
        os.mknod(train_filename)

    if not os.path.exists(validation_filename):
        os.mknod(validation_filename)

    # write to file
    fo = open(train_filename, "w")
    fo.write("\n".join(training_file_names))
    fo.close()

    fo = open(validation_filename, "w")
    fo.write("\n".join(validation_file_names))
    fo.close()

    # print process
    print("Written file is: ", train_filename, ", is_shuffle: ", args.is_shuffled)


In [ ]:
# create file list
%cd /content/Colab-edge-connect
!python prepare.py

In [ ]:
#@title Install miniconda and dependencies
%cd /content/
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
!conda install pytorch==1.1 cudatoolkit torchvision -c pytorch -y
%cd /content/Colab-edge-connect
!pip install -r requirements.txt
!conda install ipykernel -y
!pip install tensorboardX

In [ ]:
#@title Training config
%%writefile /content/training-checkpoints/config.yml
MODE: 1             # 1: train, 2: test, 3: eval
MODEL: 3            # 1: edge model, 2: inpaint model, 3: edge-inpaint model, 4: joint model
MASK: 1             # 1: random block, 2: half, 3: external, 4: (external, random block), 5: (external, random block, half)
EDGE: 1             # 1: canny, 2: external
NMS: 1              # 0: no non-max-suppression, 1: applies non-max-suppression on the external edges by multiplying by Canny
SEED: 10            # random seed
GPU: [0]            # list of gpu ids
DEBUG: 0            # turns on debugging mode
VERBOSE: 0          # turns on verbose mode in the output console
RANDOM_CROP: 1      # instead of center crop, a random crop without resize will be perofrmed
HORIZONTAL_FLIP: 1      # flip in all directions (currently not sure where and if is in the original code. I just added my flip.)
HORIZONTAL_FLIP_RATIO: 0.5
VERTICAL_FLIP: 1
VERTICAL_FLIP_RATIO: 0.5
MOSAIC_TEST: 0      

TRAIN_FLIST: /content/train/train.tflist
VAL_FLIST: /content/val/val.tflist
TEST_FLIST: /content/val/val.tflist

TRAIN_EDGE_FLIST: NULL
VAL_EDGE_FLIST: NULL
TEST_EDGE_FLIST: NULL

TRAIN_MASK_FLIST: /content/mask_train/mask_train.tflist
VAL_MASK_FLIST: /content/mask_val/mask_val.tflist
TEST_MASK_FLIST: /content/mask_val/mask_val.tflist

LR: 0.0001                    # learning rate
D2G_LR: 0.1                   # discriminator/generator learning rate ratio
BETA1: 0.0                    # adam optimizer beta1
BETA2: 0.9                    # adam optimizer beta2
BATCH_SIZE: 6                 # input batch size for training
INPUT_SIZE: 256               # input image size for training 0 for original size
SIGMA: 2                      # standard deviation of the Gaussian filter used in Canny edge detector (0: random, -1: no edge)
MAX_ITERS: 1000000                # maximum number of iterations to train the model

EDGE_THRESHOLD: 0.5           # edge detection threshold
L1_LOSS_WEIGHT: 1             # l1 loss weight
FM_LOSS_WEIGHT: 10            # feature-matching loss weight
STYLE_LOSS_WEIGHT: 250        # style loss weight
CONTENT_LOSS_WEIGHT: 0.1      # perceptual loss weight
INPAINT_ADV_LOSS_WEIGHT: 0.1  # adversarial loss weight

GAN_LOSS: nsgan               # nsgan | lsgan | hinge
GAN_POOL_SIZE: 0              # fake images pool size

SAVE_INTERVAL: 200           # how many iterations to wait before saving model (0: never)
SAMPLE_INTERVAL: 20         # how many iterations to wait before sampling (0: never)
SAMPLE_SIZE: 1               # number of images to sample
EVAL_INTERVAL: 0              # how many iterations to wait before model evaluation (0: never)
LOG_INTERVAL: 10              # how many iterations to wait before logging training status (0: never)

In [ ]:
# Train model
%cd /content/Colab-edge-connect
!python train.py --model 3 --checkpoints /content/training-checkpoints/

In [ ]:
#@title Test model
%cd /content/Colab-edge-connect
!python test.py \
  --model 3 \
  --checkpoints /content/training-checkpoints \
  --input /content/image.jpg \
  --mask /content/mask.png \
  --output /content/